# Table of contents

- [HLLE Solver](#HLLE-Solver)
- [Roe Solver](#Roe-Solver)
- [Comparison of solvers](#Comparison-of-two-approximate-solvers-with-the-exact-solution)

In the [last notebook](Euler_equations.ipynb) we studied the Euler equations of inviscid, compressible fluid flow -- including the Riemann problem and its solution.  As we saw, the exact solution of the Riemann problem is computationally expensive, since it requires solving a set of nonlinear algebraic equations.  In the context of finite volume methods, the detailed structure of the Riemann solution is almost immediately discarded -- only its impact on the neighboring cell averages is used.  So it makes sense to consider whether we can approximate the solution with less computation.  In this notebook, we investigate approximate solvers for the Euler equations.

In [ ]:
%matplotlib inline
import numpy as np
from exact_solvers import Euler
from clawpack import riemann
from utils import riemann_tools
import matplotlib.pyplot as plt
from collections import namedtuple
from ipywidgets import interact, widgets
Primitive_State = namedtuple('State', Euler.primitive_variables)
gamma = 1.4
problem_data = {}
problem_data['gamma'] = gamma
problem_data['gamma1'] = gamma - 1.0

## HLLE Solver

The HLLE solver uses only two waves with a constant state between that is uniquely defined by conservation for any choice of the two wave speeds.  The left-going wave speed is chosen to be the minimum of the Roe speed for the 1-wave and the characterstic speed $\lambda^1$ in the left state $q_\ell$.  The right-going wave speed is chosen to be the maximum of the Roe speed for the 3-wave and the characterstic speed $\lambda^3$ in the right state $q_r$.

In [ ]:
solver = riemann.euler_1D_py.euler_hll_1D

# Shock tube
left_state  = np.array(Primitive_State(Density = 3.,
                                       Velocity = -0.5,
                                       Pressure = 2.))
right_state = np.array(Primitive_State(Density = 1.,
                                       Velocity = 0.,
                                       Pressure = 1.))

print "HLL solver solution to Euler equations:"
states_hll, s_hll, hll_eval = riemann_tools.riemann_solution(solver,left_state,right_state,
                                                             problem_data=problem_data,verbose=True)
fig, ax = plt.subplots(1,3,figsize=(16,4))
riemann_tools.plot_phase(states_hll,0,1,ax[0])
riemann_tools.plot_phase(states_hll,0,2,ax[1])
riemann_tools.plot_phase(states_hll,1,2,ax[2])
riemann_tools.plot_phase_3d(states_hll)

In [ ]:
plot_function = riemann_tools.make_plot_function(states_hll,s_hll, hll_eval,
                                                 layout='vertical', conserved_variables=Euler.conserved_variables)
interact(plot_function, t=widgets.FloatSlider(min=0,max=.9,step=.1))

## Roe solver
The Roe solver is an example of a linearized Riemann solver.  It approximates the Riemann problem by considering an approximation of the flux Jacobian: $\hat{A} \approx f'(q)$ and exactly solving the Riemann problem for the linear hyperbolic system

$$q_t + \hat{A}q_x = 0.$$

The Roe linearization is chosen so that in the case that $(q_l, q_r)$ are related by a single shock, the Roe solver gives the exact solution.

In [ ]:
solver = riemann.euler_1D_py.euler_roe_1D
num_eqn = riemann.euler_1D_py.num_eqn

problem_data['efix'] = False

print "Roe solver solution to Euler equations:"
states, s, roe_eval = riemann_tools.riemann_solution(solver,left_state,right_state,
                                                     problem_data=problem_data,verbose=True)
fig, ax = plt.subplots(1,2,figsize=(10,4))
riemann_tools.plot_phase(states,0,1,ax[0])
riemann_tools.plot_phase(states,0,2,ax[1])
riemann_tools.plot_phase_3d(states)

In [ ]:
plot_function = riemann_tools.make_plot_function(states,s,roe_eval)
interact(plot_function, t=widgets.FloatSlider(min=0, max=0.9, step=0.1));

## Comparison of two approximate solvers with the exact solution


In [ ]:
ex_states, ex_speeds, reval = Euler.exact_riemann_solution(left_state ,right_state, gamma)

plot_function = riemann_tools.make_plot_function([ex_states,states_hll,states],
                                                 [ex_speeds,s_hll,s],
                                                 [reval,hll_eval,roe_eval],
                                                 ['Exact','HLLE','Roe'],
                                                 layout='vertical',conserved_variables=Euler.conserved_variables)
interact(plot_function, t=widgets.FloatSlider(min=0, max=0.9, step=0.1));

Notice that the Roe solver significantly understimates the shock speed, and even propagates the contact discontinuity in the wrong direction.  Nevertheless, when used as an ingredient in a numerical method, it gives good results.

# Comparison of full discretizations

How do these solvers impact the solution accuracy when used within a finite volume discretization?

We'll test them on the same shock tube problem considered above.

In [ ]:
from clawpack.riemann.euler_with_efix_1D_constants import density, momentum, energy, num_eqn

def setup(q_l, q_r, N=50, riemann_solver='HLL', solver_type='classic'):

    from clawpack import pyclaw
    from clawpack import riemann

    if riemann_solver == 'Roe':
        rs = riemann.euler_1D_py.euler_roe_1D
    elif riemann_solver == 'HLL':
        rs = riemann.euler_1D_py.euler_hll_1D

    if solver_type == 'classic':
        solver = pyclaw.ClawSolver1D(rs)        
    else:
        solver = pyclaw.SharpClawSolver1D(rs)

    solver.kernel_language = 'Python'
    
    solver.bc_lower[0]=pyclaw.BC.extrap
    solver.bc_upper[0]=pyclaw.BC.extrap

    x = pyclaw.Dimension(-1.0,1.0,N,name='x')
    domain = pyclaw.Domain([x])
    state = pyclaw.State(domain,num_eqn)

    gamma = 1.4
    state.problem_data['gamma']= gamma
    state.problem_data['gamma1']= gamma-1.

    state.problem_data['efix'] = False

    xc = state.grid.p_centers[0]
    
    velocity = (xc<=0)*q_l[1] + (xc>0)*q_r[1]
    pressure = (xc<=0)*q_l[2] + (xc>0)*q_r[2]

    state.q[density ,:] = (xc<=0)*q_l[0] + (xc>0)*q_r[0]
    state.q[momentum,:] = velocity * state.q[density,:]
    state.q[energy  ,:] = pressure/(gamma - 1.) + 0.5 * state.q[density,:] * velocity**2

    claw = pyclaw.Controller()
    claw.tfinal = 0.5
    claw.solution = pyclaw.Solution(state,domain)
    claw.solver = solver
    claw.num_output_times = 10
    claw.keep_copy = True
    claw.verbosity=0

    return claw

## Second-order modified Lax-Wendroff algorithm

In [ ]:
q_l = Euler.conservative_to_primitive(1.,0.,1.)
q_r = Euler.conservative_to_primitive(1./8,0.,1./10)
roe = setup(q_l,q_r,N=50,riemann_solver='Roe')
roe.run()
hll = setup(q_l,q_r,riemann_solver='HLL')
hll.run()
fine = setup(q_l,q_r,N=1000,riemann_solver='Roe')
fine.run();
xc = roe.solution.state.grid.p_centers[0]
xc_fine = fine.solution.state.grid.p_centers[0]

def plot_frame(i):
    fig, ax = plt.subplots(figsize=(12,6))
    ax.set_xlim((-1,1)); ax.set_ylim((0,1.1))
    ax.plot(xc_fine,fine.frames[i].q[density,:],'-k',lw=1)
    ax.plot(xc,hll.frames[i].q[density,:],'-ob',lw=2)
    ax.plot(xc,roe.frames[i].q[density,:],'-or',lw=2)
    plt.legend(['Fine','HLL','Roe'],loc='best')
    
interact(plot_frame, i=widgets.IntSlider(min=0, max=10, description='Frame'));

As you might expect, the HLL solver smears the middle wave (contact discontinuity) significantly more than the Roe solver does. Perhaps surprisingly, it captures the shock just as accurately as the Roe solver does.

## High-order WENO + Runge-Kutta

In [ ]:
q_l = Euler.conservative_to_primitive(1.,0.,1.)
q_r = Euler.conservative_to_primitive(1./8,0.,1./10)
roe2 = setup(q_l,q_r,N=50,riemann_solver='Roe',solver_type='sharpclaw')
roe2.run()
hll2 = setup(q_l,q_r,riemann_solver='HLL',solver_type='sharpclaw')
hll2.run()
fine2 = setup(q_l,q_r,N=1000,riemann_solver='Roe',solver_type='sharpclaw')
fine2.run();
xc = roe.solution.state.grid.p_centers[0]
xc_fine = fine.solution.state.grid.p_centers[0]

def plot_frame(i):
    fig, ax = plt.subplots(figsize=(12,6))
    ax.set_xlim((-1,1)); ax.set_ylim((0,1.1))
    ax.plot(xc_fine,fine2.frames[i].q[density,:],'-k',lw=1)
    ax.plot(xc,hll2.frames[i].q[density,:],'-ob',lw=2)
    ax.plot(xc,roe2.frames[i].q[density,:],'-or',lw=2)
    plt.legend(['Fine','HLL','Roe'],loc='best')
    
interact(plot_frame, i=widgets.IntSlider(min=0, max=10, description='Frame'));

With higher-order discretizations, the difference in the Riemann solvers is less significant.